In [66]:
import sys

import os
import pickle
# import logging

import numpy as np
import pandas as pd
import click
from typing import Union
from sklearn.metrics import precision_score
import sklearn

from msap.modeling.model_evaluation.statistics import (
    get_embedded_data,
    get_selected_features,
    get_curve_metrics,
    get_curve_metrics_test,
    get_training_statistics,
    get_baseline_training_statistics,
    get_validation_statistics,
    get_baseline_validation_statistics,
    get_testing_statistics,
    get_baseline_testing_statistics,
    get_similarity_matrix)
from msap.explanatory_analysis import get_pairwise_correlation
from msap.utils import (
    ClassifierHandler,
    load_X_and_y,
    KFold_by_feature)
from msap.utils.plot import (
    plot_heatmap,
    plot_embedded_scatter,
    plot_rfe_line,
    plot_rfe_line_detailed,
    plot_curves,
    plot_confusion_matrix)
from msap.modeling.configs import (
    ModelSelectionConfig)


METHODS_PC = ['pearson', 'spearman', 'kendall']
METHODS_EMBEDDING = ['tsne', 'pca']
METHODS_CURVE = ['pr', 'roc']
CLASSIFIER_MODES = [
    'decisiontreeclassifier',
    'gaussiannb',
    'multinomialnb',
    'svc',
    'adaboostclassifier',
    'randomforestclassifier',
    'mlpclassifier']

In [67]:
def parse_model_selection_result(ms_result: tuple) -> list:
    """Parse the model selection result tuple and get the best models.

    Args:
        ms_result: Model selection result tuple.

    Returns:
        List of best model and statistics for each classifiers.

    """
    candidates, _ = ms_result
    candidates = [(i, c, cv['best']) for i, c, cv in candidates]

    f1s_mean = []
    for i, c, cv_best in candidates:
        # Iterate over splits to calculate average F1 score.
        f1s = [cv_best[f'split_{j}']['f1'] for j in range(len(cv_best) - 1)]
        f1s_mean += [np.mean(np.nan_to_num(f1s))]

    candidates = list(zip(candidates, f1s_mean))
    candidates = sorted(candidates, key=lambda x: x[1], reverse=True) # sorts so max is first

    best_candidate_per_clf = []
    for clf in CLASSIFIER_MODES:
        for (i, c, cv_best), f1_mean in candidates:
            if c[3] == clf:
                if cv_best['param'] is not None:
                    cv_best['param'] = {k.split('__')[-1]: v
                                        for k, v in cv_best['param'].items()}

                best_candidate_per_clf += [((i, c, cv_best), f1_mean)]
                break # break to get the max

    return best_candidate_per_clf

In [68]:
# filepaths
path_input_model_selection_result = './output/pval_filter_60_MVI/output_12to18_yesmental/results.pkl'
path_input_preprocessed_data_dir = './output/pval_filter_60_MVI/output_12to18_yesmental/preprocessed'
path_output_dir = './output/pval_filter_60_MVI/output_12to18_yesmental/'
feature_label = 'y12to18_Dep_YN_216m'
random_state = 42

In [69]:
if not os.path.exists(path_output_dir):
    os.mkdir(path_output_dir)

model_selection_result = None
with open(path_input_model_selection_result, 'rb') as f:
    model_selection_result = pickle.load(f)

#print(model_selection_result)
best_candidate_per_clf = parse_model_selection_result(
    model_selection_result)
best_candidate = max(best_candidate_per_clf, key=lambda x: x[1])
_, best_combination, best_cv_result = best_candidate[0]
best_scale_mode, best_impute_mode, best_outlier_mode, best_clf \
    = best_combination

# print(best_combination)
#pd.DataFrame(best_candidate_per_clf).to_csv(
#    f"{path_output_dir}/best_clfs.csv")

In [70]:
def parse_model_selection_grid_search_results_for_best(ms_result: tuple, best_combination: tuple) -> list:
    """Parse the model selection result tuple and get the all models.

    Args:
        ms_result: Model selection result tuple.

    Returns:
        List of best model and statistics for each classifiers.

    """
    candidates, _ = ms_result
    # index, classifier, cv_result
    # no longer want best, just want all?
    candidates = [(i, c, cv) for i, c, cv in candidates]

    grid_results = []
    f1s_mean = []
    for i, c, cv in candidates:
        if c == best_combination:
            # parse every grid search result
            for key in cv:
                # Iterate over splits to calculate average F1 score for clf
                result = cv[key]
                f1s = [result[f'split_{j}']['f1'] for j in range(len(result) - 1)]
                grid_results += [(i, c, result)]
                f1s_mean += [np.mean(np.nan_to_num(f1s))]

    candidates = list(zip(grid_results, f1s_mean))
    # candidates = sorted(candidates, key=lambda x: x[1], reverse=True)

    all_candidates_of_combination = []
    for (i, c, cv), f1_mean in candidates:
        if c == best_combination:
            if cv['param'] is not None:
                # get name of parameter (last word after '__') and value
                cv['param'] = {k.split('__')[-1]: v
                                    for k, v in cv['param'].items()}

            all_candidates_of_combination += [((i, c, cv), f1_mean)]

    return all_candidates_of_combination

In [72]:
all_grid_search_results_for_best_combination = parse_model_selection_grid_search_results_for_best(
    model_selection_result, best_combination)
# all_grid_search_results_for_best_combination

[((40,
   ('standard', 'iterative', 'none', 'randomforestclassifier'),
   {'param': {'criterion': 'gini',
     'min_samples_leaf': 1,
     'min_samples_split': 2,
     'n_estimators': 25,
     'random_state': 42},
    'split_0': {'tn': 73.0,
     'fp': 116.0,
     'fn': 62.0,
     'tp': 283.0,
     'f1': 0.760752688172043},
    'split_1': {'tn': 76.0,
     'fp': 113.0,
     'fn': 60.0,
     'tp': 285.0,
     'f1': 0.7671601615074024},
    'split_2': {'tn': 77.0,
     'fp': 111.0,
     'fn': 73.0,
     'tp': 272.0,
     'f1': 0.7472527472527472},
    'split_3': {'tn': 69.0,
     'fp': 119.0,
     'fn': 52.0,
     'tp': 293.0,
     'f1': 0.7741083223249668},
    'split_4': {'tn': 75.0,
     'fp': 113.0,
     'fn': 67.0,
     'tp': 278.0,
     'f1': 0.7554347826086958}}),
  0.760941740373171),
 ((40,
   ('standard', 'iterative', 'none', 'randomforestclassifier'),
   {'param': {'criterion': 'gini',
     'min_samples_leaf': 1,
     'min_samples_split': 2,
     'n_estimators': 50,
     'rand